# Face Morphing

## Overview

This program will be divided up into parts, with the final product creating a 'morph' animation of 30fps from one face to another.

"A morph is a simultaneous warp of the image shape and a cross-dissolve of the image colors. The cross-dissolve is the easy part; controlling and doing the warp is the hard part. The warp is controlled by defining a correspondence between the two pictures. The correspondence should map eyes to eyes, mouth to mouth, chin to chin, ears to ears, etc., to get the smoothest transformations possible.
To start with, you should take a pictures of yourself on a uniform background (for instance, white). Your image should be the same size and aspect ratio as your target face (for instance, this beautiful portrait of [George](https://inst.eecs.berkeley.edu/~cs180/fa23/hw/proj3/george_small.jpg), taken by [Martin Schoeller](http://en.wikipedia.org/wiki/Martin_Schoeller)). Treat your target face as a passport photo template -- your face in the picture should be about where their face is. You can use photo editing software to resize/crop your image such that this is the case. This will make your morphing result more pleasing to the eye". - CS 180 Fall 2023

## Libraries and Universally useful functions

In [ ]:
import os
import re
import cv2
import math
import json
import glob
import random
import numpy as np
import skimage as sk
import skimage.io as skio
import matplotlib.pyplot as plt
import skimage.transform as sktr

from skimage import img_as_ubyte
from scipy.spatial import Delaunay
from skimage.draw import polygon
from PIL import Image
from sklearn.decomposition import PCA

In [ ]:
# Code for general image retrieval, storing, simple processing operations

def get_file_image(fname):
        relative_data_dir = "data"
        dir = os.getcwd()
        image_path = os.path.join(dir, relative_data_dir, fname)

        image = skio.imread(image_path)
        image = sk.img_as_float(image)
        return image

def convert_channel_to_image(r, g, b, alpha=None):
        if alpha is None:
                return np.dstack(([r, g, b]))
        return img_as_ubyte(np.dstack([r, g, b, alpha]))

def get_image_channels(image):
        assert len(image.shape) == 3
        if image.shape[2] == 3:
                return red_channel(image), green_channel(image), blue_channel(image), None
        elif image.shape[2] == 4:
                return red_channel(image), green_channel(image), blue_channel(image), alpha_channel(image)
        else:
                raise ValueError("Image does not have 3 or 4 channels")
        
def rgb2gray(image, with_alpha=False):
        assert len(image.shape) == 3
        if image.shape[2] == 3:
                return image.dot([0.2989, 0.5870, 0.1140])
        elif image.shape[2] == 4:
                rgb_channels = image[:, :, :3]
                alpha_channel = image[:, :, 3]
                
                grayed_rgb = rgb_channels.dot([0.2989, 0.5870, 0.1140])
                return grayed_rgb + (1 - alpha_channel) if with_alpha else grayed_rgb
        else:
                raise ValueError("Image does not have 3 or 4 channels")

def red_channel(image):
        return image[:, :, 0]

def green_channel(image):
        return image[:, :, 1]

def blue_channel(image):
        return image[:, :, 2]

def alpha_channel(image):
        if image.shape[2] == 4:
                return image[:, :, 3]
        else:
                return None
        
def save_image(image, name, is_out=True):
        # Scale image data to 0-255 range for storing
        normalize_factor = 1
        if image.max() <= 1.:
                normalize_factor = 255
        scaled_image = (image.copy() * normalize_factor).astype(np.uint8)
        # save the image
        file_name = 'out/' + name + '.jpg' if is_out else 'data/' + name + '.jpg'
        skio.imsave(file_name, scaled_image)
        
def get_gaussian_kernel_2d(ksize, sigma):
    k1d = cv2.getGaussianKernel(ksize=ksize, sigma=sigma)
    k2d = k1d * k1d.T
    return k2d
        
def apply_func_image(image, function):
        if len(image.shape) == 2:
                return function(image)
        r, g, b, alpha = get_image_channels(image)
        
        r = function(r)
        g = function(g)
        b = function(b)
        alpha = function(alpha) if alpha is not None else alpha
        
        return np.dstack([r, g, b]) if alpha is None else np.dstack([r, g, b, alpha])

def apply_gaussian_blurr(image, kernel_size, kernel_sigma):
        gaussian_kernel = get_gaussian_kernel_2d(kernel_size, kernel_sigma)

def normalize(image):
        min_val = np.min(image)
        max_val = np.max(image)
        normalized_image = (image - min_val) / (max_val - min_val)
        return normalized_image

def pad_image_with_white_boarders(img, top, bottom, left, right):
    height, width = img.shape[:2]
    num_channels = img.shape[2] if len(img.shape) > 2 else 1
    
    # Create white padding for each side
    left_padding = np.ones((height, left, num_channels), dtype=img.dtype)
    right_padding = np.ones((height, right, num_channels), dtype=img.dtype)
    top_padding = np.ones((top, width + left + right, num_channels), dtype=img.dtype)
    bottom_padding = np.ones((bottom, width + left + right, num_channels), dtype=img.dtype)
    
    # Combine original image and padding
    img_with_left_right_padding = np.concatenate((left_padding, img, right_padding), axis=1)
    padded_img = np.concatenate((top_padding, img_with_left_right_padding, bottom_padding), axis=0)

    return padded_img

def show_images(images, columns=None, figsize=(12, 13)):
        if columns is None:
                columns = len(images)
        rows = math.ceil((len(images) / columns))
        
        fig, axes = plt.subplots(rows, columns, figsize=figsize)

        if rows == columns and columns == 1:
                axes.imshow(images[0])                
                return
        
        for i, img in enumerate(images):
                if rows == 1:
                        axes[i].imshow(img)
                else:
                        row = i // columns
                        col = i % columns
                        axes[row][col].imshow(img)
        

def show_overlay(images, tris, columns=None, is_show_all=True, figsize=(12, 13), save_path=None):
        assert len(images) == len(tris)
        if columns is None:
                columns = len(images)
        rows = math.ceil((len(images) / columns)) + is_show_all
        fig, axes = plt.subplots(rows, columns, figsize=figsize)

        if rows == columns and columns == 1:
                axes.imshow(images[0])
                axes.plot(tris[0].points[:, 0], tris[0].points[:, 1], 'o', markersize=5, markeredgecolor='lime')
                axes.triplot(tris[0].points[:, 0], tris[0].points[:, 1], tris[0].simplices, color='lightgray')
                
                if save_path is not None:
                        axes.set_axis_off()
                        axes.margins(0,0)
                        axes.xaxis.set_major_locator(plt.NullLocator())
                        axes.yaxis.set_major_locator(plt.NullLocator())
                        extent = axes.get_window_extent().transformed(fig.dpi_scale_trans.inverted())
                        plt.savefig(save_path, bbox_inches=extent, pad_inches=0)
                return
        
        
        for i, img in enumerate(img for img in images if is_show_all):
                if rows == 1:
                        axes[i].imshow(img)
                else:
                        row = i // columns
                        col = i % columns
                        axes[row][col].imshow(img)
        
        for i, (img, tri) in enumerate(zip(images, tris)):
                index = i + is_show_all * len(images)
                if rows == 1:
                        axes[index].imshow(img)
                        axes[index].plot(tri.points[:, 0], tri.points[:, 1], 'o', markersize=5, markeredgecolor='lime')
                        axes[index].triplot(tri.points[:, 0], tri.points[:, 1], tri.simplices, color='lightgray')
                else:
                        row = index // columns
                        col = index % columns
                        axes[row][col].imshow(img)
                        axes[row][col].plot(tri.points[:, 0], tri.points[:, 1], 'o', markersize=5, markeredgecolor='lime')
                        axes[row][col].triplot(tri.points[:, 0], tri.points[:, 1], tri.simplices, color='lightgray')

        if save_path is not None:
                for i, ax in enumerate(axes.flatten()):
                        ax.set_axis_off()
                        ax.margins(0,0)
                        ax.xaxis.set_major_locator(plt.NullLocator())
                        ax.yaxis.set_major_locator(plt.NullLocator())
                        extent = ax.get_window_extent().transformed(fig.dpi_scale_trans.inverted())
                        # You should modify save_path to have a different name for each subplot
                        plt.savefig(f"{save_path[:-4]}_{i}.png", bbox_inches=extent, pad_inches=0)

## Part 1. Defining Correspondences
First, you will need to define pairs of corresponding points on the two images by hand (the more points, the better the morph, generally). The simplest way is probably to use the cpselect (matlab) tool or write your own little tool using ginput (matlab or python) and plot commands (with hold on and hold off ). In order for the morph to work you will need a consistent labeling of the two faces. So label your faces A and B in a consistent manner using the same ordering of keypoints in the two faces. It's strongly recommended that you save the points once you obtain something you are happy with so that you don't have to do all that clicking more than once!

Now, you need to provide a triangulation of these points that will be used for morphing. You can compute a triangulation any way you like, or even define it by hand. A Delaunay triangulation (see delaunay and related functions) is a good choice since it does not produce overly skinny triangles. You can compute the Delaunay triangulation on either of the point sets (but not both -- the triangulation has to be the same throughout the morph!). But the best approach would probably be to compute the triangulation at a midway shape (i.e. mean of the two point sets) to lessen the potential triangle deformations.

Note if you don't want to implement your own labeling tool, you can use [this one](https://inst.eecs.berkeley.edu/~cs194-26/fa22/upload/files/proj3/cs194-26-aex/tool.html) from a last year's student.

In [ ]:
def get_triangulation_from_correspondence_file(file_name, is_display=False, is_show_all=False, save_path=None):
    relative_data_dir = "data"
    dir = os.getcwd()
    correspondence_path = os.path.join(dir, relative_data_dir, file_name)
    
    with open(correspondence_path, 'r') as json_file:
        data = json.load(json_file)
        
    image1 = get_file_image(data['im1_name'] + ".jpg")
    image2 = get_file_image(data['im2_name'] + ".jpg")
        
    points1 = np.array(data['im1Points'])
    points2 = np.array(data['im2Points'])
    
    # Add corner points to make wrapping easier
    new_points = np.array([[0, 0], [image1.shape[1] - 1, 0], [0, image1.shape[0] - 1], [image1.shape[1] - 1, image1.shape[0] - 1]])
    points1 = np.vstack((points1, new_points))
    
    new_points = np.array([[0, 0], [image2.shape[1] - 1, 0], [0, image2.shape[0] - 1], [image2.shape[1] - 1, image2.shape[0] - 1]])
    points2 = np.vstack((points2, new_points))
    
    tri1 = Delaunay(points1)
    tri2 = Delaunay(points2)
    tri2.simplices = tri1.simplices.copy()  # Copy the triangle information
    
    if is_display:
        show_overlay(images=[image1, image2], tris=[tri1, tri2], is_show_all=is_show_all, save_path=save_path)
    return image1, image2, tri1, tri2

tofu_img, crisp_img, tofu_tri, crisp_tri = get_triangulation_from_correspondence_file('tofu_with_chains_crisp.json', True, True, save_path='out/overlay_images/tofu_crisp')

In [ ]:
me_img, dominic_fike_img, me_tri, dominic_fike_tri = get_triangulation_from_correspondence_file('me_dominic_fike.json', True, True, save_path='out/overlay_images/me_dominic_fike')

In [ ]:
me_img, kaiona_img, me_tri, kaiona_tri = get_triangulation_from_correspondence_file('me_kaiona.json', True, True, save_path='out/overlay_images/me_kaiona')

## Part 2. Computing the "Mid-way Face"

In this part, we will compute the "Mid-way Face" between the first image to the final image. First, we will find the transitional image between the first and final image by taking a weighted average. Then, we will find the affine transformation matrix from both images to the transitional image. Finally, we will use the new found affine transformation matrix to perform wrapping and color dissolution.


To find the affine transformation function, we can use the corresponding points taken during Part 1 to solve for the affine transformation matrix. Since there are 6 unknowns, we will need 3 points (Note: Each point has an x and y component). After finding the affine transformation matrix, we can use inverse wrapping to take pixels from our first image into our new "Mid-way Face" image.

Affine Transformation with Homogenous coordinates:
$$
\begin{bmatrix}
   x' \\
   y' \\
   1 \\
\end{bmatrix}
=
\begin{bmatrix}
   a & b & c \\
   d & e & f \\
   0 & 0 & 1
\end{bmatrix}
\begin{bmatrix}
   x \\
   y \\
   1 \\
\end{bmatrix}
$$

Solving for Affine Transformation through Correspondence points:
<div style="text-align: center;">

In linear algebra:

x<sub>1</sub>' = ax<sub>1</sub> + by<sub>1</sub> + c \
x<sub>2</sub>' = ax<sub>2</sub> + by<sub>2</sub> + c \
x<sub>3</sub>' = ax<sub>3</sub> + by<sub>3</sub> + c

y<sub>1</sub>' = dx<sub>1</sub> + ey<sub>1</sub> + f \
y<sub>2</sub>' = dx<sub>2</sub> + ey<sub>2</sub> + f \
y<sub>3</sub>' = dx<sub>3</sub> + ey<sub>3</sub> + f 

Using matrix calculations:

$$
\begin{bmatrix}
   a & b & c \\
   d & e & f \\
   0 & 0 & 1
\end{bmatrix}
\begin{bmatrix}
   x_{1} & x_{2} & x_{3} \\
   y_{1} & y_{2} & y_{3} \\
   1 & 1 & 1 \\
\end{bmatrix}
=
\begin{bmatrix}
   x_{1}' & x_{2}' & x_{3}' \\
   y_{1}' & y_{2}' & y_{3}' \\
   1 & 1 & 1\\
\end{bmatrix}
$$
$$
\begin{bmatrix}
   a & b & c \\
   d & e & f \\
   0 & 0 & 1 \\ 
\end{bmatrix}
=
\begin{bmatrix}
   x_{1}' & x_{2}' & x_{3}' \\
   y_{1}' & y_{2}' & y_{3}' \\
   1 & 1 & 1\\
\end{bmatrix}
\begin{bmatrix}
   x_{1} & x_{2} & x_{3} \\
   y_{1} & y_{2} & y_{3} \\
   1 & 1 & 1 \\
\end{bmatrix}^{-1}
$$

<br>
<div style="text-align: left;">
Note: Unlike conventional matrix calculations, the image representations in python are (y, x) where y represents the vertical value and x represents the horrizontal value. However, this does not affect our calculations since the rows in the affine matrix is correspondingly flipped as well!

$$
\begin{bmatrix}
   a & b & c \\
   d & e & f \\
   0 & 0 & 1
\end{bmatrix}
\begin{bmatrix}
   y_{1} & y_{2} & y_{3} \\
   x_{1} & x_{2} & x_{3} \\
   1 & 1 & 1 \\
\end{bmatrix}
=
\begin{bmatrix}
   y_{1}' & y_{2}' & y_{3}' \\
   x_{1}' & x_{2}' & x_{3}' \\
   1 & 1 & 1\\
\end{bmatrix}
$$
$$
\begin{bmatrix}
   a & b & c \\
   d & e & f \\
   0 & 0 & 1 \\ 
\end{bmatrix}
=
\begin{bmatrix}
   y_{1}' & y_{2}' & y_{3}' \\
   x_{1}' & x_{2}' & x_{3}' \\
   1 & 1 & 1\\
\end{bmatrix}
\begin{bmatrix}
   y_{1} & y_{2} & y_{3} \\
   x_{1} & x_{2} & x_{3} \\
   1 & 1 & 1 \\
\end{bmatrix}^{-1}
$$

In [ ]:
def compute_affine_function(src_pts, dest_pts):
    """_summary_

    Args:
        src_pts (_type_): _description_
        dest_pts (_type_): _description_

    Returns:
        _type_: _description_
    """
    assert type(src_pts) == type(dest_pts)
    
    if not isinstance(src_pts, np.ndarray):
        assert len(src_pts) >= 3 and len(dest_pts) == len(src_pts)
        assert len(src_pts[0]) == 2 and len(dest_pts[0]) == 2
        
        src_pts = np.array(src_pts)
        dest_pts = np.array(dest_pts)
        
    assert src_pts.shape == dest_pts.shape
    ones_row = np.ones((1, src_pts.shape[0]))
    
    src_pts = np.vstack((src_pts.T, ones_row))
    dest_pts = np.vstack((dest_pts.T, ones_row))
    src_pts_inverse = np.linalg.inv(src_pts)
    
    return np.dot(dest_pts, src_pts_inverse)

def get_shape_size(pts):
    """_summary_

    Args:
        pts (_type_): _description_

    Returns:
        _type_: _description_
    """
    max_y = np.max(pts[:, 1])
    max_x = np.max(pts[:, 0])
    
    height = math.ceil(max_y) + 1
    width = math.ceil(max_x) + 1

    return height, width

def get_triangle_coords(triangle_vertices):
    """_summary_

    Args:
        triangle_vertices (_type_): _description_

    Returns:
        _type_: _description_
    """
    return polygon(triangle_vertices[:, 1], triangle_vertices[:, 0])
    

def get_transition_img(img_shape, src_tri, dest_tri, wrap_frac=0.5):
    """_summary_

    Args:
        src_tri (_type_): _description_
        dest_tri (_type_): _description_
        wrap_frac (float, optional): _description_. Defaults to 0.5.

    Returns:
        _type_: _description_
    """
    transition_tri = Delaunay((1-wrap_frac) * src_tri.points + wrap_frac * dest_tri.points)
    transition_tri.simplices = src_tri.simplices.copy()
    transition_image = np.zeros(img_shape)
    
    return transition_image, transition_tri

def inverse_warping(src_img, dest_img, result_im, src_pts, dest_pts, transition_pts, dissolve_frac=0.5): 
    """Performs inverse wrapping within the triangle points.

    Args:
        src_img (_type_): The image that we want to morph from
        dest_img (_type_): The final image that we want to morph into
        result_im (_type_): The current image that we are morphing into
        src_pts (_type_): The points in the beginning image
        dest_pts (_type_): The points in the final image
        transition_pts (_type_): The points we are currently morphing into
        dissolve_frac (float, optional): The degree of color dissolution we want. Defaults to 0.5.

    Returns:
        _type_: _description_
    """
    src_affine_matrix = compute_affine_function(transition_pts, src_pts)
    dest_affine_matrix = compute_affine_function(transition_pts, dest_pts)
    
    transition_y, transition_x = get_triangle_coords(transition_pts)
    homogenous_transition_coords = np.vstack([transition_x, transition_y, np.ones_like(transition_y)])
    homogenous_src_coords = np.dot(src_affine_matrix, homogenous_transition_coords)
    homogenous_dest_coords = np.dot(dest_affine_matrix, homogenous_transition_coords)
    
    src_x, src_y = homogenous_src_coords[0, :], homogenous_src_coords[1, :]
    dest_x, dest_y = homogenous_dest_coords[0, :], homogenous_dest_coords[1, :]
    
    src_x, src_y = np.round(src_x).astype(int), np.round(src_y).astype(int)
    dest_x, dest_y = np.round(dest_x).astype(int), np.round(dest_y).astype(int)
    
    in_bounds_indexes = (
        (0 <= src_x) & (src_x < src_img.shape[1]) &
        (0 <= src_y) & (src_y < src_img.shape[0]) &
        (0 <= dest_x) & (dest_x < dest_img.shape[1]) &
        (0 <= dest_y) & (dest_y < dest_img.shape[0]) &
        (0 <= transition_y) & (transition_y < result_im.shape[0]) &
        (0 <= transition_x) & (transition_x < result_im.shape[1])
    )
    
    transition_y, transition_x = transition_y[in_bounds_indexes], transition_x[in_bounds_indexes]
    src_x, src_y = src_x[in_bounds_indexes], src_y[in_bounds_indexes]
    dest_x, dest_y = dest_x[in_bounds_indexes], dest_y[in_bounds_indexes]
    
    result_im[transition_y, transition_x, :] = (
        (1 - dissolve_frac) * src_img[src_y, src_x, :] + 
        dissolve_frac * dest_img[dest_y, dest_x, :]
    )
    
    return result_im

def morph(src_img, dest_img, src_tri, dest_tri, transition_im=None, transition_tri=None, wrap_frac=0.5, dissolve_frac=0.5):
    """_summary_

    Args:
        src_img (_type_): _description_
        dest_img (_type_): _description_
        src_tri (_type_): _description_
        dest_tri (_type_): _description_
        transition_im (_type_, optional): _description_. Defaults to None.
        transition_tri (_type_, optional): _description_. Defaults to None.
        wrap_frac (float, optional): _description_. Defaults to 0.5.
        dissolve_frac (float, optional): _description_. Defaults to 0.5.

    Returns:
        _type_: _description_
    """
    if transition_im is None:
        transition_im, transition_tri = get_transition_img(src_img.shape, src_tri, dest_tri, wrap_frac)
    
    for simplex in src_tri.simplices:
        src_pts = src_tri.points[simplex]
        dest_pts = dest_tri.points[simplex]
        transition_pts = transition_tri.points[simplex]
        
        transition_im = inverse_warping(src_img, dest_img, transition_im, src_pts, dest_pts, transition_pts, dissolve_frac)
    
    normalized_transition_im = np.clip(normalize(transition_im), 0, 1)
    
    return normalized_transition_im, transition_tri    
        
# Show results
tofu_img, crisp_img, tofu_tri, crisp_tri = get_triangulation_from_correspondence_file('tofu_with_chains_crisp.json', False)
tofu_crisp_transition_image, tofu_crisp_transition_tri = morph(tofu_img, crisp_img, tofu_tri, crisp_tri)

images = [tofu_img, tofu_crisp_transition_image, crisp_img]
tris = [tofu_tri, tofu_crisp_transition_tri, crisp_tri]

os.makedirs('out/overlay_images/tofu_crisp_transition', exist_ok=True)
show_overlay(images, tris, columns=3, is_show_all=True, save_path='out/overlay_images/tofu_crisp_transition')
# show_overlay(images, tris, columns=3, is_show_all=True)



In [ ]:
me_img, dominic_fike_img, me_tri, dominic_fike_tri = get_triangulation_from_correspondence_file('me_dominic_fike.json', False)
me_dominic_fike_transition_image, me_dominic_fike_transition_tri = morph(me_img, dominic_fike_img, me_tri, dominic_fike_tri)

images = [me_img, me_dominic_fike_transition_image, dominic_fike_img]
tris = [me_tri, me_dominic_fike_transition_tri, dominic_fike_tri]

os.makedirs('out/overlay_images/me_dominic_transition', exist_ok=True)
show_overlay(images, tris, columns=3, is_show_all=True, save_path='out/overlay_images/me_dominic_transition')
# show_overlay(images, tris, columns=3, is_show_all=True)
save_image(me_dominic_fike_transition_image, "me_dominic_fike_combine_image")

In [ ]:
me_img, kaiona_img, me_tri, kaiona_tri = get_triangulation_from_correspondence_file('me_kaiona.json', False)
me_kaiona_transition_image, me_kaiona_transition_tri = morph(me_img, kaiona_img, me_tri, kaiona_tri)

images = [me_img, me_kaiona_transition_image, kaiona_img]
tris = [me_tri, me_kaiona_transition_tri, dominic_fike_tri]

os.makedirs('out/overlay_images/me_kaiona_transition', exist_ok=True)
show_overlay(images, tris, columns=3, is_show_all=True, save_path='out/overlay_images/me_kaiona_transition')
# show_overlay(images, tris, columns=3, is_show_all=True)

## Part 3 The Morph Sequence

In [ ]:
def create_gif_from_images(image_list, gif_name='animated.gif', duration=None):
    """_summary_

    Args:
        image_list (_type_): _description_
        gif_name (str, optional): _description_. Defaults to 'animated.gif'.
    """
    os.makedirs('out/gif_folder', exist_ok=True)
    os.makedirs('out/image_folder', exist_ok=True)
    pil_images = []
    
    if duration is None:
        duration = len(image_list) // 30
    
    for i, img in enumerate(image_list):
        if img.dtype == np.float32 or img.dtype == np.float64:
            img = (img * 255).astype('uint8')
        
        img_name = gif_name[:-4]
        img_path = os.path.join('out/image_folder', f'{img_name}_{i}.jpg')
        
        # Save the image with skimage.io
        skio.imsave(img_path, img)
        
        img_pil = Image.open(img_path)
        pil_images.append(img_pil)

    # Save images as a GIF
    gif_path = os.path.join('out/gif_folder', gif_name)
    pil_images[0].save(gif_path, save_all=True, append_images=pil_images[1:], loop=0, duration=duration)

In [ ]:
def get_morph_sequence_from_file(file_name, num_sequence=45, is_save_images=True):
    """_summary_

    Args:
        file_name (_type_): _description_
        num_sequence (int, optional): _description_. Defaults to 45.
        is_save_images (bool, optional): _description_. Defaults to True.

    Returns:
        _type_: _description_
    """
    assert num_sequence > 0
    src_img, dest_img, src_tri, dest_tri = get_triangulation_from_correspondence_file(file_name)
    wrap_frac=0
    dissolve_frac=0
    
    frac_growth_rate = 1 / num_sequence
    
    morph_sequence = [src_img]
    morph_tris = [src_tri]
    
    for i in range(num_sequence):
        wrap_frac += float(frac_growth_rate)
        dissolve_frac += float(frac_growth_rate)
        transitional_im, transitional_tri = morph(src_img, dest_img, src_tri, dest_tri, wrap_frac=wrap_frac, dissolve_frac=wrap_frac)
        
        morph_sequence.append(transitional_im)
        morph_tris.append(transitional_tri)
    
        
    morph_sequence.append(dest_img)
    morph_tris.append(dest_tri)
    if is_save_images:
        print("Saving images. Do not turn off power.")
        create_gif_from_images(morph_sequence, file_name[:-5]+'.gif')
        print("Images saved.")
    return morph_sequence, morph_tris

# Uncomment to get the gifs
get_morph_sequence_from_file('tofu_with_chains_crisp.json')
get_morph_sequence_from_file('me_dominic_fike.json')
get_morph_sequence_from_file('me_kaiona.json')


## Part 4. The "Mean face" of a population

In [ ]:
# Class to help store the points and image from danes dataset. Allows better readability
# Goal: Have one class that stores the image and their corresponding data points
# Note: Since the image file name is already stored in the asf file, we only need to pass in the asf file.
#       To ensure that the background is also added when morphing, I manually harded the corners into points
def read_asf_file(fname):
    if fname[-4:] != '.asf':
        fname += '.asf'
    relative_data_dir = "data/danes/"
    dir = os.getcwd()
    file_path = os.path.join(dir, relative_data_dir, fname)
    
    with open(file_path, 'r') as file:
        content = file.read()
    return content

class DanesDataset:
    def __init__(self):
        self.danes_images = {}
        self.get_data()
        
    def get_data(self):
        file_pattern = '*.asf' 
        full_pattern = "data/danes/" + file_pattern
        file_list = glob.glob(full_pattern)  

        for file_path in file_list:
            fname = os.path.basename(file_path)
            self.danes_images[int(fname[:2])] = self.DanesImage(fname)
            
    def get_image(self, im_num):
        return self.danes_images[im_num].im
    
    def show_overlay(self, im_num):
        self.danes_images[im_num].show_overlay()
    
    def get_im_pts_tri(self, im_num):
        dan_im = self.danes_images[im_num]
        return dan_im.im, dan_im.get_coords(), dan_im.get_tri()
    
    def get_all_im_pts_tri(self):
        img_list = []
        pts = []
        tris = []
        
        for index in self.danes_images.keys():
            img, pt, tri = self.get_im_pts_tri(index)
            img_list.append(img)
            pts.append(pt)
            tris.append(tri)
            
        # make sure all the tris get the same triangles to avoid folding
        for i in range(len(tris)):
            tris[i].simplices = tris[0].simplices.copy()
            
        return img_list, pts, tris

    
    class DanesImage:
        def __init__(self, fname):
            self.im = None
            self.data_pts = {}
            self.parse_danes_asf_file(fname)
        
        def parse_danes_asf_file(self, fname):
            lines = read_asf_file(fname)            
            points_pattern = re.compile(r'(\d+)\s+(\d+)\s+(-?\d+\.\d+)\s+(-?\d+\.\d+)\s+(\d+)\s+(\d+)\s+(\d+)')
            
            # format: <path#> <type> <x rel.> <y rel.> <point#> <connects from> <connects to>
            for pt in points_pattern.findall(lines):
                assert len(pt) == 7
                self.data_pts[int(pt[4])] = self.Datapoint(*pt)
            image_fname_pattern = re.compile(r'.+\.bmp')
            im_fname = image_fname_pattern.search(lines).group()
            self.im = get_file_image("Danes/"+im_fname)
            
            # Manually added corners as points to help with wrapping.
            
            self.data_pts[-1] = self.Datapoint("-1", "-1", "0", "0", "-1", "-1", "-1")
            self.data_pts[-2] = self.Datapoint("-2", "-1", "0", "1", "-2", "-2", "-2")
            self.data_pts[-3] = self.Datapoint("-3", "-1", "1", "0", "-3", "-3", "-3")
            self.data_pts[-4] = self.Datapoint("-4", "-1", "1", "1", "-4", "-4", "-4")
            
        
        def get_coord(self, pt_num):
            return self.data_pts[pt_num].get_coord(self.im)
        
        def get_coords(self):
            coords = []
            
            for pt in self.data_pts.values():
                x, y = pt.get_rel_coord()
                coords.append([x, y])
            
            coords = np.array(coords)
            coords[:, 0] *= self.im.shape[1]
            coords[:, 1] *= self.im.shape[0]
            return coords
        
        def get_tri(self):
            pts = self.get_coords()
            return Delaunay(pts)
        
        def show_overlay(self):
            pts = self.get_coords()
            tri = self.get_tri()
            fig, axes = plt.subplots(1, 1, figsize=(10, 10))
            axes.imshow(self.im)
            axes.plot(pts[:, 0], pts[:, 1], 'o', markersize=5, markeredgecolor='lime')
            axes.triplot(tri.points[:, 0], tri.points[:, 1], tri.simplices, color='lightgray')
            
        
        class Datapoint:
            def __init__(self, path_num, pt_type, relative_x, relative_y, point_num, connect_from, connect_to):
                self.path_num =int(path_num)
                self.type = int(pt_type)
                self.relative_x = float(relative_x)
                self.relative_y = float(relative_y)
                self.point_num = int(point_num) # Unique ID of point, store here for debugging for now
                self.connect_from = int(connect_from)
                self.connect_to = int(connect_to)
            
            def get_coord(self, im):
                x = round(self.relative_x * im.shape[1])
                y = round(self.relative_y * im.shape[0])
                
                return x, y
            
            def get_rel_coord(self):
                return self.relative_x, self.relative_y

In [ ]:
def get_average_transition_im(im_shape, src_pts_list, src_tri_list):
    """_summary_

    Args:
        src_pts_list (_type_): _description_
        src_tri_list (_type_): _description_

    Returns:
        _type_: _description_
    """
    avg_pts = np.round(np.mean(np.array(src_pts_list), axis = 0))
    avg_tri = Delaunay(avg_pts)
    avg_tri.simplices = src_tri_list[0].simplices.copy()
    
    avg_image = np.zeros(im_shape)
    
    # avg_image = np.zeros((*get_shape_size(avg_pts), 3))
    
    return avg_image, avg_tri

def compute_inverse_coords(src_pts, result_pts):
    """_summary_

    Args:
        src_pts (_type_): _description_
        result_pts (_type_): _description_

    Returns:
        _type_: _description_
    """
    result_y_in_tri, result_x_in_tri = get_triangle_coords(result_pts)
    homogenous_result_coords = np.vstack([result_x_in_tri, result_y_in_tri, np.ones_like(result_y_in_tri)])
    
    affine_matrix = compute_affine_function(src_pts, result_pts)
    affine_matrix = np.linalg.inv(affine_matrix)
    
    homogenous_img_coords = np.dot(affine_matrix, homogenous_result_coords)
    img_x, img_y = homogenous_img_coords[0, :], homogenous_img_coords[1, :]

    return img_y, img_x, result_y_in_tri, result_x_in_tri

def bilinear_interpolation(image, y, x):
    """_summary_

    Args:
        image (_type_): _description_
        y (_type_): _description_
        x (_type_): _description_

    Returns:
        _type_: _description_
    """
    y_floor = np.floor(y).astype(int)
    x_floor = np.floor(x).astype(int)
    
    in_bounds_indexes = (
        (0 <= x_floor) & (x_floor + 1 < image.shape[1]) &
        (0 <= y_floor) & (y_floor + 1 < image.shape[0])
    )
    
    y_floor = y_floor[in_bounds_indexes]
    x_floor = x_floor[in_bounds_indexes]
    
    # Also filter y and x with the in_bounds_indexes
    y = y[in_bounds_indexes]
    x = x[in_bounds_indexes]
    
    
    value_top_left = image[y_floor, x_floor, :]
    value_top_right = image[y_floor, x_floor + 1, :]
    value_bottom_left = image[y_floor + 1, x_floor, :]
    value_bottom_right = image[y_floor + 1, x_floor + 1, :]
    
    # For weight calculation
    x_relative = (x - x_floor).reshape(-1, 1)
    y_relative = (y - y_floor).reshape(-1, 1)
    
    top_left_weight = (1 - x_relative) * (1 - y_relative)
    top_right_weight = (1 - x_relative) * y_relative
    bottom_left_weight = x_relative * (1 - y_relative)
    bottom_right_weight =  x_relative * y_relative
    
    value_top_left = top_left_weight * value_top_left
    value_top_right = top_right_weight * value_top_right
    value_bottom_left = bottom_left_weight * value_bottom_left
    value_bottom_right = bottom_right_weight * value_bottom_right
    
    return value_top_left + value_top_right + value_bottom_left + value_bottom_right, in_bounds_indexes

def nearest_neighbor(image, y, x):
    """_summary_

    Args:
        image (_type_): _description_
        y (_type_): _description_
        x (_type_): _description_

    Returns:
        _type_: _description_
    """
    img_y, img_x = np.round(y).astype(int), np.round(x).astype(int)
    in_bounds_indexes = (
        (0 <= img_x) & (img_x < image.shape[1]) &
        (0 <= img_y) & (img_y < image.shape[0]) 
    )
    img_y, img_x = img_y[in_bounds_indexes], img_x[in_bounds_indexes]
    
    return image[img_y, img_x, :], in_bounds_indexes
    

def running_average_inverse_wrapping(img, result_im, tris, result_tris, running_count=0):
    """_summary_

    Args:
        img (_type_): _description_
        result_im (_type_): _description_
        tris (_type_): _description_
        result_tris (_type_): _description_
        running_count (int, optional): _description_. Defaults to 0.

    Returns:
        _type_: _description_
    """
    average_frac = 1 / (running_count + 1)
    
    temp_result_im = np.zeros_like(result_im)
    count_matrix = np.zeros_like(result_im, dtype=int)
    for simplex in tris.simplices:
        pts = tris.points[simplex]
        result_pts = result_tris.points[simplex]

        img_y, img_x, result_y_in_tri, result_x_in_tri = compute_inverse_coords(pts, result_pts)
        
        interpolated_value, in_bounds_indexes = bilinear_interpolation(img, img_y, img_x)
        result_y_in_tri, result_x_in_tri = result_y_in_tri[in_bounds_indexes], result_x_in_tri[in_bounds_indexes]
        temp_result_im[result_y_in_tri, result_x_in_tri, :] += interpolated_value
        
        count_matrix[result_y_in_tri, result_x_in_tri, :] += 1
        
    overlapping_areas = count_matrix > 1
    temp_result_im[overlapping_areas] /= count_matrix[overlapping_areas]
    result_im = (1 - average_frac) * result_im + average_frac * temp_result_im
    
    return result_im
        

def average_morph(img_list, img_pts_list, img_tris_list):
    """_summary_

    Args:
        img_list (_type_): _description_
        img_pts_list (_type_): _description_
        img_tris_list (_type_): _description_

    Returns:
        _type_: _description_
    """
    avg_im, avg_tri = get_average_transition_im(img_list[0].shape, img_pts_list, img_tris_list)
        
    for i, (img, tri) in enumerate(zip(img_list, img_tris_list)):
        avg_im = running_average_inverse_wrapping(img, avg_im, tri, avg_tri, i)
        
    # avg_im = normalize(avg_im / len(img_list))
    return normalize(avg_im), avg_tri


danes_im_list, danes_pts_list, danes_tris_list = DanesDataset().get_all_im_pts_tri()
avg_danes_im, avg_danes_tri = average_morph(danes_im_list, danes_pts_list, danes_tris_list)
skio.imshow(avg_danes_im)
skio.show()
save_image(avg_danes_im, "average_face_in_danes_dataset")

In [ ]:
danes_morph_to_average_image = []
danes_morph_to_average_tris = []
danes_im_list, danes_pts_list, danes_tris_list = DanesDataset().get_all_im_pts_tri()
avg_danes_im, avg_danes_tri = average_morph(danes_im_list, danes_pts_list, danes_tris_list)

os.makedirs('out/morphed_danes_to_average', exist_ok=True)
for i, (img, pts, tris) in enumerate(zip(danes_im_list, danes_pts_list, danes_tris_list)):
    morphed_im, morphed_tris = morph(img, avg_danes_im, tris, avg_danes_tri, wrap_frac=0.5, dissolve_frac=0.5)
    danes_morph_to_average_image.append(morphed_im)
    danes_morph_to_average_tris.append(morphed_tris)
    
    save_image(morphed_im, "morphed_danes_to_average/image{}".format(i))  
    
show_images(danes_morph_to_average_image, columns=8, figsize=(22, 23))

In [ ]:
danes_wrapped_to_average_image = []
danes_wrapped_to_average_tris = []
danes_im_list, danes_pts_list, danes_tris_list = DanesDataset().get_all_im_pts_tri()
avg_danes_im, avg_danes_tri = average_morph(danes_im_list, danes_pts_list, danes_tris_list)

os.makedirs('out/wrapped_danes_to_average', exist_ok=True)
for i, (img, pts, tris) in enumerate(zip(danes_im_list, danes_pts_list, danes_tris_list)):
    wrapped_im, wrapped_tris = morph(img, avg_danes_im, tris, avg_danes_tri, wrap_frac=1, dissolve_frac=0)
    danes_wrapped_to_average_image.append(wrapped_im)
    danes_wrapped_to_average_tris.append(wrapped_tris)
    
    save_image(wrapped_im, "wrapped_danes_to_average/image{}".format(i))  
    
show_images(danes_wrapped_to_average_image, columns=8, figsize=(22, 23))

In [ ]:
me_img, average_face_in_danes_img, me_tri, average_face_in_danes_tri = get_triangulation_from_correspondence_file('me_average_face_in_danes_dataset_for_morphing_white_background.json')
morphed_me_avg_dane_im, morphed_me_avg_dane_tri = morph(me_img, average_face_in_danes_img, me_tri, average_face_in_danes_tri, wrap_frac=0.5, dissolve_frac=0.5)
show_overlay([me_img, morphed_me_avg_dane_im, average_face_in_danes_img], [me_tri, morphed_me_avg_dane_tri, average_face_in_danes_tri], columns=3, is_show_all=True)
save_image(morphed_me_avg_dane_im, 'me_average_face_in_danes_dataset_for_morphing_white_background')

In [ ]:
me_img, average_face_in_danes_img, me_tri, average_face_in_danes_tri = get_triangulation_from_correspondence_file('me_average_face_in_danes_dataset_for_morphing_white_background.json')
morphed_me_avg_dane_im, morphed_me_avg_dane_tri = morph(me_img, average_face_in_danes_img, me_tri, average_face_in_danes_tri, wrap_frac=0, dissolve_frac=0.5)
show_overlay([me_img, morphed_me_avg_dane_im, average_face_in_danes_img], [me_tri, morphed_me_avg_dane_tri, average_face_in_danes_tri], columns=3, is_show_all=True)
save_image(morphed_me_avg_dane_im, 'me_average_danes_color_only')

In [ ]:
me_img, average_face_in_danes_img, me_tri, average_face_in_danes_tri = get_triangulation_from_correspondence_file('me_average_face_in_danes_dataset_for_morphing_white_background.json')
morphed_me_avg_dane_im, morphed_me_avg_dane_tri = morph(me_img, average_face_in_danes_img, me_tri, average_face_in_danes_tri, wrap_frac=0.5, dissolve_frac=0)
show_overlay([me_img, morphed_me_avg_dane_im, average_face_in_danes_img], [me_tri, morphed_me_avg_dane_tri, average_face_in_danes_tri], columns=3, is_show_all=True)
save_image(morphed_me_avg_dane_im, 'me_average_danes_wrapping_only')

In [ ]:
get_morph_sequence_from_file('me_average_face_in_danes_dataset_for_morphing_white_background.json', num_sequence=45, is_save_images=True)

## Part 5. Caricatures: Extrapolating from the mean

In [ ]:
me_img, average_face_in_danes_img, me_tri, average_face_in_danes_tri = get_triangulation_from_correspondence_file('me_average_face_in_danes_dataset_for_morphing_white_background.json')
morphed_me_avg_dane_im, morphed_me_avg_dane_tri = morph(me_img, average_face_in_danes_img, me_tri, average_face_in_danes_tri, wrap_frac=-0.2, dissolve_frac=0)
show_overlay([me_img, morphed_me_avg_dane_im, average_face_in_danes_img], [me_tri, morphed_me_avg_dane_tri, average_face_in_danes_tri], columns=3, is_show_all=True)
save_image(morphed_me_avg_dane_im, 'me_extrapolated_average_face_in_danes_alpha_0.2')

In [ ]:
me_img, average_face_in_danes_img, me_tri, average_face_in_danes_tri = get_triangulation_from_correspondence_file('me_average_face_in_danes_dataset_for_morphing_white_background.json')
morphed_me_avg_dane_im, morphed_me_avg_dane_tri = morph(me_img, average_face_in_danes_img, me_tri, average_face_in_danes_tri, wrap_frac=-0.4, dissolve_frac=0)
show_overlay([me_img, morphed_me_avg_dane_im, average_face_in_danes_img], [me_tri, morphed_me_avg_dane_tri, average_face_in_danes_tri], columns=3, is_show_all=True)
save_image(morphed_me_avg_dane_im, 'me_extrapolated_average_face_in_danes_alpha_0.4')

In [ ]:
me_img, average_face_in_danes_img, me_tri, average_face_in_danes_tri = get_triangulation_from_correspondence_file('me_average_face_in_danes_dataset_for_morphing_white_background.json')
morphed_me_avg_dane_im, morphed_me_avg_dane_tri = morph(me_img, average_face_in_danes_img, me_tri, average_face_in_danes_tri, wrap_frac=-0.6, dissolve_frac=0)
show_overlay([me_img, morphed_me_avg_dane_im, average_face_in_danes_img], [me_tri, morphed_me_avg_dane_tri, average_face_in_danes_tri], columns=3, is_show_all=True)
save_image(morphed_me_avg_dane_im, 'me_extrapolated_average_face_in_danes_alpha_0.6')

In [ ]:
danes_im_list, danes_pts_list, danes_tris_list = DanesDataset().get_all_im_pts_tri()
avg_danes_im, avg_danes_tri = average_morph(danes_im_list, danes_pts_list, danes_tris_list)
morphed_first_danes__avg_dane_im_alpha_point_2, morphed_first_danes_avg_dane_tri_2 = morph(
    danes_im_list[0], 
    average_face_in_danes_img, 
    danes_tris_list[0], avg_danes_tri,
    wrap_frac=-0.6, dissolve_frac=0
)
morphed_first_danes__avg_dane_im_alpha_point_4, morphed_first_danes_avg_dane_tri_4 = morph(
    danes_im_list[0], 
    average_face_in_danes_img,  
    danes_tris_list[0], avg_danes_tri,
    wrap_frac=-1.2, dissolve_frac=0
)
morphed_first_danes__avg_dane_im_alpha_point_6, morphed_first_danes_avg_dane_tri_6 = morph(
    danes_im_list[0], 
    average_face_in_danes_img,  
    danes_tris_list[0], avg_danes_tri,
    wrap_frac=-1.8, dissolve_frac=0
)

show_overlay(
    [morphed_first_danes__avg_dane_im_alpha_point_2, morphed_first_danes__avg_dane_im_alpha_point_4, morphed_first_danes__avg_dane_im_alpha_point_6], 
    [morphed_first_danes_avg_dane_tri_2, morphed_first_danes_avg_dane_tri_4, morphed_first_danes_avg_dane_tri_6], 
    columns=3, is_show_all=True
)
save_image(morphed_first_danes__avg_dane_im_alpha_point_2, 'morphed_first_danes__avg_dane_im_alpha_point_0.6')
save_image(morphed_first_danes__avg_dane_im_alpha_point_4, 'morphed_first_danes__avg_dane_im_alpha_point_1.2')
save_image(morphed_first_danes__avg_dane_im_alpha_point_6, 'morphed_first_danes__avg_dane_im_alpha_point_1.8')



In [ ]:
# Extract the average faces from the compilation back to individual images
average_female_oriented_ordering = [
    'Uzbek', 'Welsh', 'West African', 'Vietnamese', 'Chinese', 'Hungarian', 'Japanese', 'Korean', 'Puerto Rican',
    'Thai', 'African American', 'Afghan', 'Central Africa', 'Burmese', 'Cambodian', 'English', 'Ethiopian', 'Filipino'
    'Finnish', 'French', 'German', 'Greek', 'Indian', 'Iranian', 'Irish','Israeli', 'Italian',
    'Mexican', 'Latvian (Lithuanian)', 'Mongolain', 'Peruvian', 'Polish', 'Romanian', 'Russian', 'Samoan', 'South African',
    'South Indian', 'Spaniard', 'Swedish', 'Swiss', 'Taiwanese'
]

average_male_oriented_ordering = [
    'Austria', 'Afghanistan', 'Argentina', 'Burma (Myanmar)', 'Germany', 'Greece', 'India', 'Iran',
    'Cambodia', 'England', 'Ethiopia', 'France', 'Iraq', 'Ireland', 'Israel', 'Mexico',
    'Mongolia', 'Peru', 'Poland', 'Puerto Rico', 'Uzbekistan', 'African America', 'White America', 'China',
    'Romania', 'Russia', 'Samoa', 'Saudi Arabia', 'Czech Republic', 'Philippines', 'Hungary', 'Italy',
    'Serbia', 'South Africa', 'South India', 'Spain', 'Japan', 'Korea', 'Thailand', 'Brazil',
    'Switzerland', 'Taiwan', 'Tibet', 'Ukraine', 'Vietnam', 'West Africa'
]

# row x col
average_female_oriented_shape = (5, 9)
average_male_oriented_shape = (6, 8)

# amount of boundary in each image, (start_y, start_x, end_y, end_x)
average_female_oriented_boundary = (0, 0, 40, 0)
average_male_oriented_boundary = (5, 1, 15, 1)

# file names
average_female_oriented_fname = 'average_faces_around_the_world/female_oriented/averageface.jpg'
average_male_oriented_fname = 'average_faces_around_the_world/male_oriented/average_male_faces_around_the_world.jpg'

def get_image_average_faces(image_number, sex_orientation):
    if sex_orientation == 0:
        image_country_ordering = average_female_oriented_ordering
        all_image_shape = average_female_oriented_shape
        remove_boundary = average_female_oriented_boundary
        all_image = get_file_image(average_female_oriented_fname)
    else:
        image_country_ordering = average_male_oriented_ordering
        all_image_shape = average_male_oriented_shape
        remove_boundary = average_male_oriented_boundary
        all_image = get_file_image(average_male_oriented_fname)
    
    assert image_number < len(image_country_ordering)
    image_name = image_country_ordering[image_number]
    
    image_size = (all_image.shape[0] // all_image_shape[0]), all_image.shape[1] // all_image_shape[1]
    
    row_num = image_number // all_image_shape[1]
    col_num = image_number % all_image_shape[1]
    
    start_image_y, start_image_x = row_num * image_size[0] + remove_boundary[0], col_num * image_size[1] + remove_boundary[1]
    end_image_y, end_image_x = max(0, start_image_y + image_size[0] - remove_boundary[2]), max(0, start_image_x + image_size[1] - remove_boundary[3])
    return all_image[start_image_y:end_image_y, start_image_x:end_image_x, :], image_name
    
    
# Save all image for post processing
os.makedirs('out/average_male_oriented_face', exist_ok=True)
for i in range(len(average_female_oriented_ordering)):
    image, image_name = get_image_average_faces(i, 0)
    save_image(image, 'average_female_oriented_face/' + image_name)

for i in range(len(average_male_oriented_ordering)):
    image, image_name = get_image_average_faces(i, 1)
    save_image(image, 'average_male_oriented_face/' + image_name)


In [ ]:
me_img, average_peru_male_oriented_img, me_tri, average_peru_male_tri = get_triangulation_from_correspondence_file('me_Peru.json', False)
me_peru_image, me_peru_tri = morph(me_img, average_peru_male_oriented_img, me_tri, average_peru_male_tri, wrap_frac=0, dissolve_frac=0.5)
images = [me_img, me_peru_image, average_peru_male_oriented_img]
tris = [me_tri, me_peru_tri, average_peru_male_tri]

show_overlay(images, tris, columns=3, is_show_all=True)
save_image(me_peru_image, "me_peru_image_color_only")

In [ ]:
me_img, average_peru_male_oriented_img, me_tri, average_peru_male_tri = get_triangulation_from_correspondence_file('me_Peru.json', False)
me_peru_image, me_peru_tri = morph(me_img, average_peru_male_oriented_img, me_tri, average_peru_male_tri, wrap_frac=0.5, dissolve_frac=0)
images = [me_img, me_peru_image, average_peru_male_oriented_img]
tris = [me_tri, me_peru_tri, average_peru_male_tri]

show_overlay(images, tris, columns=3, is_show_all=True)
save_image(me_peru_image, "me_peru_image_shape_only")

In [ ]:
me_img, average_peru_male_oriented_img, me_tri, average_peru_male_tri = get_triangulation_from_correspondence_file('me_Peru.json', False)
me_peru_image, me_peru_tri = morph(me_img, average_peru_male_oriented_img, me_tri, average_peru_male_tri, wrap_frac=0.5, dissolve_frac=0.5)
images = [me_img, me_peru_image, average_peru_male_oriented_img]
tris = [me_tri, me_peru_tri, average_peru_male_tri]

show_overlay(images, tris, columns=3, is_show_all=True)
save_image(me_peru_image, "me_peru_image")

In [ ]:
me_img, average_ethiopia_female_oriented_img, me_tri, average_ethiopia_female_tri = get_triangulation_from_correspondence_file('me_Ethiopian.json', False)
me_ethiopia_image, me_ethiopia_tri = morph(me_img, average_ethiopia_female_oriented_img, me_tri, average_ethiopia_female_tri, wrap_frac=0, dissolve_frac=0.5)
images = [me_img, me_ethiopia_image, average_ethiopia_female_oriented_img]
tris = [me_tri, me_ethiopia_tri, average_ethiopia_female_tri]

show_overlay(images, tris, columns=3, is_show_all=True)
save_image(me_ethiopia_image, "me_ethiopia_image_color_only")

In [ ]:
me_img, average_ethiopia_female_oriented_img, me_tri, average_ethiopia_female_tri = get_triangulation_from_correspondence_file('me_Ethiopian.json', False)
me_ethiopia_image, me_ethiopia_tri = morph(me_img, average_ethiopia_female_oriented_img, me_tri, average_ethiopia_female_tri, wrap_frac=0.5, dissolve_frac=0)
images = [me_img, me_ethiopia_image, average_ethiopia_female_oriented_img]
tris = [me_tri, me_ethiopia_tri, average_ethiopia_female_tri]

show_overlay(images, tris, columns=3, is_show_all=True)
save_image(me_ethiopia_image, "me_ethiopia_image_shape_only")

In [ ]:
me_img, average_ethiopia_female_oriented_img, me_tri, average_ethiopia_female_tri = get_triangulation_from_correspondence_file('me_Ethiopian.json', False)
me_ethiopia_image, me_ethiopia_tri = morph(me_img, average_ethiopia_female_oriented_img, me_tri, average_ethiopia_female_tri, wrap_frac=0.5, dissolve_frac=0.5)
images = [me_img, me_ethiopia_image, average_ethiopia_female_oriented_img]
tris = [me_tri, me_ethiopia_tri, average_ethiopia_female_tri]

show_overlay(images, tris, columns=3, is_show_all=True)
save_image(me_ethiopia_image, "me_ethiopia_image")

In [ ]:
me_img, average_ethiopia_female_oriented_img, me_tri, average_ethiopia_female_tri = get_triangulation_from_correspondence_file('me_Ethiopian.json', False)
me_ethiopia_image, me_ethiopia_tri = morph(me_img, average_ethiopia_female_oriented_img, me_tri, average_ethiopia_female_tri, wrap_frac=1, dissolve_frac=0)
images = [me_img, me_ethiopia_image, average_ethiopia_female_oriented_img]
tris = [me_tri, me_ethiopia_tri, average_ethiopia_female_tri]

show_overlay(images, tris, columns=3, is_show_all=True)
save_image(me_ethiopia_image, "caricatures_me_ethiopia_image")

In [ ]:
def get_eigen_face(img_list, img_pts_list, img_tris_list, mean_image=None, mean_tri=None, num_eigenfaces=None):
    if mean_image is None or mean_tri is None:
        mean_image = np.mean(img_list)
    
    face_shape = img_list[0].shape
    face_matrix = []
    
    # Aligning facial landmarks to mean and flatten them
    for img, tri in zip(img_list, img_tris_list):
        assert img.shape == face_shape
        aligned_img, _ = morph(img, mean_image, tri, mean_tri, wrap_frac=1, dissolve_frac=0)
        centered_im = aligned_img - mean_image
        face_matrix.append(centered_im.flatten())
        
    face_matrix = np.array(face_matrix)
    pca = PCA().fit(face_matrix)
    
    eigenfaces = pca.components_
    
    if num_eigenfaces is not None:
        eigenfaces = eigenfaces[:num_eigenfaces]
    # eigenfaces = [(eigenface - eigenface.min()) / (eigenface.max() - eigenface.min()) for eigenface in eigenfaces] # Normalizing eigenfaces
    
    return np.array([eigenface.reshape(face_shape) for eigenface in eigenfaces])

def create_caricature(target_img, mean_img, target_tri, mean_tri, eigenfaces, exaggeration_factor_list=None):
    if exaggeration_factor_list is None:
        exaggeration_factor_list = np.ones(eigenfaces.shape[0])
        
    aligned_img, _ = morph(target_img, mean_img, target_tri, mean_tri, wrap_frac=1, dissolve_frac=0)
    
    # Compute coefficients
    centered_target = aligned_img.flatten() - mean_img.flatten()
    coefficients = [np.dot(centered_target, eigenface.flatten()) for eigenface in eigenfaces]

    # Exaggerate coefficients
    exaggerated_coefficients = [coeff * exaggeration_factor for coeff, exaggeration_factor in zip(coefficients, exaggeration_factor_list)]
    
    
    # Reconstruct the caricature image
    caricature = mean_img.flatten().astype(float)
    for coeff, eigenface in zip(exaggerated_coefficients, eigenfaces):
        caricature += coeff * eigenface.flatten()
    caricature = caricature.reshape(aligned_img.shape)
    
    # Normalize the caricature if needed
    caricature = (caricature - caricature.min()) / (caricature.max() - caricature.min())

    return caricature

danes_im_list, danes_pts_list, danes_tris_list = DanesDataset().get_all_im_pts_tri()
avg_danes_im, avg_danes_tri = average_morph(danes_im_list, danes_pts_list, danes_tris_list)
danes_eigenfaces = get_eigen_face(danes_im_list, danes_pts_list, danes_tris_list, avg_danes_im, avg_danes_tri)

display_eigenfaces = [(eigenface - eigenface.min()) / (eigenface.max() - eigenface.min()) for eigenface in danes_eigenfaces]

show_images(display_eigenfaces, columns=8, figsize=(35, 21))
        
os.makedirs('out/danes_eigen_faces', exist_ok=True)
for i, display_eigenface in enumerate(display_eigenfaces):
    img_name = 'danes_eigen_face'
    img_path = os.path.join('danes_eigen_faces', f'{img_name}_{i}')
    save_image(display_eigenface, img_path)
    

In [ ]:
limited_danes_eigenfaces = danes_eigenfaces[:1]
exaggeration_factor_list = [1] * danes_eigenfaces.shape[0]
reconstruct_image_im = create_caricature(avg_danes_im, avg_danes_im, avg_danes_tri, avg_danes_tri, limited_danes_eigenfaces, exaggeration_factor_list)
skio.imshow(reconstruct_image_im)
skio.show()

os.makedirs('out/danes_caricatures/average_reconstructions', exist_ok=True)
for i in range(1, min(len(danes_eigenfaces), 10)):
    img_name = 'average_reconstruct_image_with_num_eigen_no_exaggeration'
    img_path = os.path.join('danes_caricatures/average_reconstructions', f'{img_name}_{i}')
    
    limited_danes_eigenfaces = danes_eigenfaces[:i]
    exaggeration_factor_list = [1] * danes_eigenfaces.shape[0]
    reconstruct_image_im = create_caricature(avg_danes_im, avg_danes_im, avg_danes_tri, avg_danes_tri, limited_danes_eigenfaces, exaggeration_factor_list)
    save_image(reconstruct_image_im, img_path)

In [ ]:
limited_danes_eigenfaces = danes_eigenfaces[:10]
exaggeration_factor_list = [1] * limited_danes_eigenfaces.shape[0]
caricature_im = create_caricature(danes_im_list[0], avg_danes_im, danes_tris_list[0], avg_danes_tri, limited_danes_eigenfaces, exaggeration_factor_list)
skio.imshow(caricature_im)
skio.show()

caricature_list = []
os.makedirs('out/danes_caricatures/first_danes_image_reconstruction', exist_ok=True)
for i in range(1, min(len(danes_eigenfaces), 100)):
    img_name = 'danes_reconstruct_first_image_with_num_eigen_no_exaggeration'
    img_path = os.path.join('danes_caricatures/first_danes_image_reconstruction', f'{img_name}_{i}')
    
    limited_danes_eigenfaces = danes_eigenfaces[:i]
    exaggeration_factor_list = [1] * limited_danes_eigenfaces.shape[0]
    reconstruct_image_im = create_caricature(danes_im_list[0], avg_danes_im, danes_tris_list[0], avg_danes_tri, limited_danes_eigenfaces, exaggeration_factor_list)
    save_image(reconstruct_image_im, img_path)
    caricature_list.append(reconstruct_image_im)
    
create_gif_from_images(caricature_list, 'danes_reconstruct_first_image_with_num_eigen_no_exaggeration.gif', duration=len(caricature_list) * 8)
show_images(caricature_list, columns=6, figsize=(30, 30))

In [ ]:
os.makedirs('out/danes_caricatures/first_danish_carticulation', exist_ok=True)

caricature_list = []
for i in range(1, min(len(danes_eigenfaces), 300)):
    img_name = 'danes_reconstruct_first_image_with_random_exageration_num_eigenvalues'
    img_path = os.path.join('danes_caricatures/first_danish_carticulation', f'{img_name}_{i}')
    
    limited_danes_eigenfaces = danes_eigenfaces[:]
    exaggeration_factor_list = np.random.rand(limited_danes_eigenfaces.shape[0])
    caricature_image_im = create_caricature(danes_im_list[0], avg_danes_im, danes_tris_list[0], avg_danes_tri, limited_danes_eigenfaces, exaggeration_factor_list)
    save_image(caricature_image_im, img_path)
    caricature_list.append(caricature_image_im)
    
create_gif_from_images(caricature_list, 'danes_reconstruct_first_image_with_random_exageration_num_eigenvalues.gif',duration=len(caricature_list) * 10)
show_images(caricature_list, columns=6, figsize=(30, 30))
